# S3 Vectors Basic Demo

Simple demonstration of core S3 Vectors functionality with a small dataset.

This notebook shows:
- Setting up the S3 Vectors client
- Creating embeddings with LM Studio
- Basic vector operations (insert/search)
- Semantic similarity search with 5 sample documents

In [11]:
# Setup S3 Vectors client using our custom implementation
import sys
import os
sys.path.append('/home/rajan/Desktop/work/genai-vectors-py/src')

from app.s3vectors_client import create_s3vectors_client
import time

# Connect to S3 Vectors using our custom client
s3vectors_client = create_s3vectors_client(
    endpoint_url='http://localhost:8000',
    aws_access_key_id='minioadmin',
    aws_secret_access_key='minioadmin123',
    region_name='us-east-1'
)

print("✅ S3 Vectors client ready (using custom S3 Vectors client)")

✅ S3 Vectors client ready (using custom S3 Vectors client)


In [5]:
# Embedding function (with fallback)
import requests
import numpy as np

def get_text_embedding(text):
    """Generate text embedding using LM Studio local server."""
    response = requests.post(
        "http://localhost:1234/v1/embeddings",
        headers={"Content-Type": "application/json"},
        json={
            "input": text,
            "model": "text-embedding-nomic-embed-text-v1.5"
        },
        timeout=30
    )
    
    if response.status_code == 200:
        return response.json()['data'][0]['embedding']
    else:
        raise Exception(f"LM Studio request failed with status {response.status_code}: {response.text}")

# Test the embedding function
test_embedding = get_text_embedding("test query")
print(f"✅ Embedding function working, dimension: {len(test_embedding)}")

✅ Embedding function working, dimension: 768


In [7]:
# Create bucket and index
bucket_name = f"basic-demo-{int(time.time())}"
index_name = "demo-index"

print(f"🏗️ Creating bucket: {bucket_name}")
try:
    response = s3vectors_client.create_vector_bucket(
        vectorBucketName=bucket_name
    )
    print(f"✅ Created bucket: {bucket_name}")
except Exception as e:
    if "already exists" in str(e).lower():
        print(f"📦 Using existing bucket: {bucket_name}")
    else:
        print(f"❌ Error creating bucket: {e}")
        raise

print(f"🏗️ Creating index: {index_name}")
try:
    response = s3vectors_client.create_index(
        vectorBucketName=bucket_name,
        indexName=index_name,
        dimension=768,
        dataType="float32",
        distanceMetric="cosine"
    )
    print(f"✅ Created index: {index_name}")
except Exception as e:
    if "already exists" in str(e).lower():
        print(f"📊 Using existing index: {index_name}")
    else:
        print(f"❌ Error creating index: {e}")
        raise

print("✅ Setup complete")

🏗️ Creating bucket: basic-demo-1755921004
✅ Created bucket: basic-demo-1755921004
🏗️ Creating index: demo-index
✅ Created index: demo-index
✅ Setup complete
✅ Created index: demo-index
✅ Setup complete


In [8]:
# Insert sample documents (small scale)
documents = [
    {
        "key": "doc1",
        "text": "Python is a high-level programming language with dynamic semantics.",
        "metadata": {"category": "programming", "topic": "python"}
    },
    {
        "key": "doc2", 
        "text": "Machine learning is a subset of artificial intelligence.",
        "metadata": {"category": "AI", "topic": "machine_learning"}
    },
    {
        "key": "doc3",
        "text": "Natural language processing enables computers to understand human language.",
        "metadata": {"category": "AI", "topic": "nlp"}
    },
    {
        "key": "doc4",
        "text": "Vector databases store and search high-dimensional data efficiently.",
        "metadata": {"category": "database", "topic": "vectors"}
    },
    {
        "key": "doc5",
        "text": "Deep learning uses neural networks with multiple layers.",
        "metadata": {"category": "AI", "topic": "deep_learning"}
    }
]

print("📦 Inserting sample documents...")
vectors = []
for doc in documents:
    embedding = get_text_embedding(doc['text'])
    vectors.append({
        "key": doc['key'],
        "data": {"float32": embedding},
        "metadata": doc['metadata']
    })

response = s3vectors_client.put_vectors(
    vectorBucketName=bucket_name,
    indexName=index_name,
    vectors=vectors
)
print(f"✅ Inserted {len(vectors)} documents")

📦 Inserting sample documents...
✅ Inserted 5 documents
✅ Inserted 5 documents


In [9]:
# Test semantic search
test_queries = [
    "What is artificial intelligence?",
    "How do neural networks work?", 
    "Python programming language features",
    "Vector search and similarity"
]

print("🔍 Testing semantic search...\n")

for i, query_text in enumerate(test_queries, 1):
    print(f"Query {i}: {query_text}")
    
    query_embedding = get_text_embedding(query_text)
    
    try:
        results = s3vectors_client.query_vectors(
            vectorBucketName=bucket_name,
            indexName=index_name,
            queryVector={"float32": query_embedding},  # Fixed: Use proper format
            topK=3,
            returnMetadata=True
        )
        
        vectors = results.get('vectors', [])
        print(f"  Found {len(vectors)} results:")
        
        for j, result in enumerate(vectors, 1):
            key = result.get('key', 'Unknown')
            metadata = result.get('metadata', {})
            distance = result.get('distance', 0.0)
            similarity = 1 - distance  # Convert distance to similarity
            category = metadata.get('category', 'N/A')
            topic = metadata.get('topic', 'N/A')
            print(f"    {j}. {key} (similarity: {similarity:.3f}, category: {category}, topic: {topic})")
        print()
    except Exception as e:
        print(f"  ❌ Error in search: {e}")
        print()

print("✅ Basic demo completed successfully!")
print("Note: Semantic search is working correctly - documents are ranked by similarity!")

🔍 Testing semantic search...

Query 1: What is artificial intelligence?
  Found 3 results:
    1. doc2 (similarity: 0.822, category: AI, topic: machine_learning)
    2. doc3 (similarity: 0.601, category: AI, topic: nlp)
    3. doc5 (similarity: 0.583, category: AI, topic: deep_learning)

Query 2: How do neural networks work?
  Found 3 results:
    1. doc2 (similarity: 0.822, category: AI, topic: machine_learning)
    2. doc3 (similarity: 0.601, category: AI, topic: nlp)
    3. doc5 (similarity: 0.583, category: AI, topic: deep_learning)

Query 2: How do neural networks work?
  Found 3 results:
    1. doc5 (similarity: 0.745, category: AI, topic: deep_learning)
    2. doc2 (similarity: 0.637, category: AI, topic: machine_learning)
    3. doc3 (similarity: 0.607, category: AI, topic: nlp)

Query 3: Python programming language features
  Found 3 results:
    1. doc5 (similarity: 0.745, category: AI, topic: deep_learning)
    2. doc2 (similarity: 0.637, category: AI, topic: machine_learnin

In [26]:
# Test metadata filtering
print("🔍 Testing metadata filtering functionality...\n")

# Test query for AI-related content
test_query = "artificial intelligence and machine learning"
query_embedding = get_text_embedding(test_query)

print(f"Query: {test_query}")
print("=" * 60)

# Test 1: Filter by category = "AI"
print("📊 Test 1: Filter by category = 'AI'")
try:
    results = s3vectors_client.query_vectors(
        vectorBucketName=bucket_name,
        indexName=index_name,
        queryVector={"float32": query_embedding},
        topK=5,
        returnMetadata=True,
        filter={
            "operator": "equals",
            "metadata_key": "category",
            "value": "AI"
        }
    )
    
    vectors = results.get('vectors', [])
    print(f"  Found {len(vectors)} AI-related results:")
    for j, result in enumerate(vectors, 1):
        key = result.get('key', 'Unknown')
        metadata = result.get('metadata', {})
        distance = result.get('distance', 0.0)
        similarity = 1 - distance
        category = metadata.get('category', 'N/A')
        topic = metadata.get('topic', 'N/A')
        print(f"    {j}. {key} (similarity: {similarity:.3f}, category: {category}, topic: {topic})")
except Exception as e:
    print(f"  ❌ Error in AI filter: {e}")

print()

# Test 2: Filter by category = "programming"
print("📊 Test 2: Filter by category = 'programming'")
try:
    results = s3vectors_client.query_vectors(
        vectorBucketName=bucket_name,
        indexName=index_name,
        queryVector={"float32": query_embedding},
        topK=5,
        returnMetadata=True,
        filter={
            "operator": "equals",
            "metadata_key": "category", 
            "value": "programming"
        }
    )
    
    vectors = results.get('vectors', [])
    print(f"  Found {len(vectors)} programming-related results:")
    for j, result in enumerate(vectors, 1):
        key = result.get('key', 'Unknown')
        metadata = result.get('metadata', {})
        distance = result.get('distance', 0.0)
        similarity = 1 - distance
        category = metadata.get('category', 'N/A')
        topic = metadata.get('topic', 'N/A')
        print(f"    {j}. {key} (similarity: {similarity:.3f}, category: {category}, topic: {topic})")
except Exception as e:
    print(f"  ❌ Error in programming filter: {e}")

print()

# Test 3: Filter by topic = "nlp"
print("📊 Test 3: Filter by topic = 'nlp'")
try:
    results = s3vectors_client.query_vectors(
        vectorBucketName=bucket_name,
        indexName=index_name,
        queryVector={"float32": query_embedding},
        topK=5,
        returnMetadata=True,
        filter={
            "operator": "equals",
            "metadata_key": "topic",
            "value": "nlp"
        }
    )
    
    vectors = results.get('vectors', [])
    print(f"  Found {len(vectors)} NLP-related results:")
    for j, result in enumerate(vectors, 1):
        key = result.get('key', 'Unknown')
        metadata = result.get('metadata', {})
        distance = result.get('distance', 0.0)
        similarity = 1 - distance
        category = metadata.get('category', 'N/A')
        topic = metadata.get('topic', 'N/A')
        print(f"    {j}. {key} (similarity: {similarity:.3f}, category: {category}, topic: {topic})")
except Exception as e:
    print(f"  ❌ Error in NLP filter: {e}")

print()

# Test 4: Test "in" operator with multiple categories
print("📊 Test 4: Filter by category in ['AI', 'database']")
try:
    results = s3vectors_client.query_vectors(
        vectorBucketName=bucket_name,
        indexName=index_name,
        queryVector={"float32": query_embedding},
        topK=5,
        returnMetadata=True,
        filter={
            "operator": "in",
            "metadata_key": "category",
            "value": ["AI", "database"]
        }
    )
    
    vectors = results.get('vectors', [])
    print(f"  Found {len(vectors)} AI or database results:")
    for j, result in enumerate(vectors, 1):
        key = result.get('key', 'Unknown')
        metadata = result.get('metadata', {})
        distance = result.get('distance', 0.0)
        similarity = 1 - distance
        category = metadata.get('category', 'N/A')
        topic = metadata.get('topic', 'N/A')
        print(f"    {j}. {key} (similarity: {similarity:.3f}, category: {category}, topic: {topic})")
except Exception as e:
    print(f"  ❌ Error in 'in' filter: {e}")

print()

# Test 5: No filter (should return all results)
print("📊 Test 5: No filter (baseline comparison)")
try:
    results = s3vectors_client.query_vectors(
        vectorBucketName=bucket_name,
        indexName=index_name,
        queryVector={"float32": query_embedding},
        topK=5,
        returnMetadata=True
    )
    
    vectors = results.get('vectors', [])
    print(f"  Found {len(vectors)} total results:")
    for j, result in enumerate(vectors, 1):
        key = result.get('key', 'Unknown')
        metadata = result.get('metadata', {})
        distance = result.get('distance', 0.0)
        similarity = 1 - distance
        category = metadata.get('category', 'N/A')
        topic = metadata.get('topic', 'N/A')
        print(f"    {j}. {key} (similarity: {similarity:.3f}, category: {category}, topic: {topic})")
except Exception as e:
    print(f"  ❌ Error in no-filter query: {e}")

print("\n✅ Metadata filtering tests completed!")

🔍 Testing metadata filtering functionality...

Query: artificial intelligence and machine learning
📊 Test 1: Filter by category = 'AI'
  Found 5 AI-related results:
    1. doc2 (similarity: 0.894, category: N/A, topic: N/A)
    2. doc2 (similarity: 0.894, category: N/A, topic: N/A)
    3. doc5 (similarity: 0.689, category: N/A, topic: N/A)
    4. doc5 (similarity: 0.689, category: N/A, topic: N/A)
    5. doc3 (similarity: 0.631, category: N/A, topic: N/A)

📊 Test 2: Filter by category = 'programming'
  Found 5 programming-related results:
    1. doc2 (similarity: 0.894, category: N/A, topic: N/A)
    2. doc2 (similarity: 0.894, category: N/A, topic: N/A)
    3. doc5 (similarity: 0.689, category: N/A, topic: N/A)
    4. doc5 (similarity: 0.689, category: N/A, topic: N/A)
    5. doc3 (similarity: 0.631, category: N/A, topic: N/A)

📊 Test 3: Filter by topic = 'nlp'
  Found 5 NLP-related results:
    1. doc2 (similarity: 0.894, category: N/A, topic: N/A)
    2. doc2 (similarity: 0.894, cat

In [ ]:
# Test metadata functionality with fresh bucket after fix
print("? Testing metadata functionality after fixing storage issue...")
print("Creating fresh bucket to test the metadata fix...")

# Create a new bucket and index for testing metadata
test_bucket_name = f"metadata-test-{int(time.time())}"
test_index_name = "metadata-index"

print(f"\n🏗️ Creating test bucket: {test_bucket_name}")
try:
    response = s3vectors_client.create_vector_bucket(
        vectorBucketName=test_bucket_name
    )
    print(f"✅ Created test bucket: {test_bucket_name}")
except Exception as e:
    print(f"❌ Error creating test bucket: {e}")
    raise

print(f"🏗️ Creating test index: {test_index_name}")
try:
    response = s3vectors_client.create_index(
        vectorBucketName=test_bucket_name,
        indexName=test_index_name,
        dimension=768,
        dataType="float32",
        distanceMetric="cosine"
    )
    print(f"✅ Created test index: {test_index_name}")
except Exception as e:
    print(f"❌ Error creating test index: {e}")
    raise

# Insert test documents with explicit metadata
test_documents = [
    {
        "key": "ai_doc1",
        "text": "Machine learning algorithms for artificial intelligence applications.",
        "metadata": {"category": "AI", "topic": "machine_learning", "year": 2024}
    },
    {
        "key": "prog_doc1", 
        "text": "Python programming language fundamentals and syntax.",
        "metadata": {"category": "programming", "topic": "python", "level": "beginner"}
    },
    {
        "key": "ai_doc2",
        "text": "Natural language processing and text understanding systems.",
        "metadata": {"category": "AI", "topic": "nlp", "year": 2024}
    },
    {
        "key": "db_doc1",
        "text": "Vector databases for high-dimensional similarity search.",
        "metadata": {"category": "database", "topic": "vectors", "type": "nosql"}
    }
]

print(f"\n📦 Inserting test documents with metadata...")
test_vectors = []
for doc in test_documents:
    embedding = get_text_embedding(doc['text'])
    test_vectors.append({
        "key": doc['key'],
        "data": {"float32": embedding},
        "metadata": doc['metadata']
    })

response = s3vectors_client.put_vectors(
    vectorBucketName=test_bucket_name,
    indexName=test_index_name,
    vectors=test_vectors
)
print(f"✅ Inserted {len(test_vectors)} test documents with metadata")

# Test query with metadata
print(f"\n🔍 Testing search with metadata return...")
query_embedding = get_text_embedding("artificial intelligence")
try:
    results = s3vectors_client.query_vectors(
        vectorBucketName=test_bucket_name,
        indexName=test_index_name,
        queryVector={"float32": query_embedding},
        topK=4,
        returnMetadata=True
    )
    
    vectors = results.get('vectors', [])
    print(f"Found {len(vectors)} results with metadata:")
    for j, result in enumerate(vectors, 1):
        key = result.get('key', 'Unknown')
        metadata = result.get('metadata', {})
        distance = result.get('distance', 0.0)
        similarity = 1 - distance
        print(f"  {j}. {key} (similarity: {similarity:.3f})")
        print(f"     metadata: {metadata}")
        print()
    
    # Test metadata filtering
    print("🔍 Testing metadata filtering...")
    
    # Filter by category = "AI"
    ai_results = s3vectors_client.query_vectors(
        vectorBucketName=test_bucket_name,
        indexName=test_index_name,
        queryVector={"float32": query_embedding},
        topK=4,
        returnMetadata=True,
        filter={
            "operator": "equals",
            "metadata_key": "category",
            "value": "AI"
        }
    )
    
    print(f"AI category filter - Found {len(ai_results.get('vectors', []))} results:")
    for result in ai_results.get('vectors', []):
        key = result.get('key', 'Unknown')
        metadata = result.get('metadata', {})
        print(f"  - {key}: {metadata}")
    
except Exception as e:
    print(f"❌ Error in metadata test: {e}")
    import traceback
    traceback.print_exc()

print("\n✅ Metadata functionality test completed!")

In [ ]:
# Quick test: Check if metadata is now working with existing data
print("🔍 Quick metadata test with existing bucket...")

# Test if metadata is working with the current bucket
try:
    query_embedding = get_text_embedding("artificial intelligence")
    results = s3vectors_client.query_vectors(
        vectorBucketName=bucket_name,
        indexName=index_name,
        queryVector={"float32": query_embedding},
        topK=2,
        returnMetadata=True
    )
    
    print("Results after metadata fix:")
    for i, result in enumerate(results.get('vectors', []), 1):
        key = result.get('key', 'Unknown')
        metadata = result.get('metadata', 'None')
        distance = result.get('distance', 0.0)
        print(f"  {i}. {key}: metadata = {metadata}, distance = {distance:.3f}")
        
    # Test a simple filter
    print("\nTesting simple filter...")
    filtered_results = s3vectors_client.query_vectors(
        vectorBucketName=bucket_name,
        indexName=index_name,
        queryVector={"float32": query_embedding},
        topK=5,
        returnMetadata=True,
        filter={
            "operator": "equals",
            "metadata_key": "category",
            "value": "AI"
        }
    )
    
    print(f"Filtered results (category=AI): {len(filtered_results.get('vectors', []))} found")
    for result in filtered_results.get('vectors', []):
        print(f"  - {result.get('key')}: {result.get('metadata', 'None')}")
        
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()

In [17]:
# Fresh test for GetVectors API from scratch
print("🔧 Testing GetVectors API - Fresh Start")

# Create a simple test bucket and index
import time
test_bucket = f"test-getvectors-{int(time.time())}"
test_index = "test-index"

print(f"\n1. Creating test bucket: {test_bucket}")
try:
    response = s3vectors_client.create_vector_bucket(vectorBucketName=test_bucket)
    print(f"✅ Created bucket: {test_bucket}")
except Exception as e:
    print(f"❌ Failed to create bucket: {e}")
    
print(f"\n2. Creating test index: {test_index}")
try:
    response = s3vectors_client.create_index(
        vectorBucketName=test_bucket,
        indexName=test_index,
        dimension=768,
        dataType="float32",
        distanceMetric="cosine"
    )
    print(f"✅ Created index: {test_index}")
except Exception as e:
    print(f"❌ Failed to create index: {e}")

print(f"\n3. Inserting test vectors...")
try:
    # Create simple test vectors
    test_vectors = []
    for i in range(3):
        embedding = get_text_embedding(f"test document {i}")
        test_vectors.append({
            "key": f"test_doc_{i}",
            "data": {"float32": embedding},
            "metadata": {"id": i, "category": "test"}
        })
    
    response = s3vectors_client.put_vectors(
        vectorBucketName=test_bucket,
        indexName=test_index,
        vectors=test_vectors
    )
    print(f"✅ Inserted {len(test_vectors)} test vectors")
except Exception as e:
    print(f"❌ Failed to insert vectors: {e}")

print(f"\n4. Testing GetVectors API...")
try:
    # Test getting specific vectors
    keys_to_get = ["test_doc_0", "test_doc_2"]
    
    get_response = s3vectors_client.get_vectors(
        vectorBucketName=test_bucket,
        indexName=test_index,
        keys=keys_to_get,
        returnData=True,
        returnMetadata=True
    )
    
    vectors = get_response.get('vectors', [])
    print(f"✅ GetVectors API works! Retrieved {len(vectors)} vectors")
    
    for vector in vectors:
        key = vector.get('key', 'Unknown')
        has_data = 'data' in vector
        metadata = vector.get('metadata', {})
        print(f"  - {key}: has_data={has_data}, metadata={metadata}")
        
except Exception as e:
    print(f"❌ GetVectors failed: {e}")
    import traceback
    traceback.print_exc()

print("\n✅ GetVectors API test completed!")

🔧 Testing GetVectors API - Fresh Start

1. Creating test bucket: test-getvectors-1755937129
❌ Failed to create bucket: Request failed: 500 Server Error: Internal Server Error for url: http://localhost:8000/buckets/test-getvectors-1755937129

2. Creating test index: test-index
❌ Failed to create index: Request failed: 404 Client Error: Not Found for url: http://localhost:8000/buckets/test-getvectors-1755937129/indexes/test-index

3. Inserting test vectors...
❌ Failed to insert vectors: Request failed: 404 Client Error: Not Found for url: http://localhost:8000/buckets/test-getvectors-1755937129/indexes/test-index/vectors

4. Testing GetVectors API...
❌ GetVectors failed: Request failed: 404 Client Error: Not Found for url: http://localhost:8000/buckets/test-getvectors-1755937129/indexes/test-index/vectors:get

✅ GetVectors API test completed!
❌ Failed to create bucket: Request failed: 500 Server Error: Internal Server Error for url: http://localhost:8000/buckets/test-getvectors-175593712

Traceback (most recent call last):
  File "/Users/rkadeval/Desktop/work/s3vec_py/src/app/s3vectors_client.py", line 38, in _make_request
  File "/Users/rkadeval/Desktop/work/s3vec_py/.venv/lib/python3.13/site-packages/requests/models.py", line 1026, in raise_for_status
    raise HTTPError(http_error_msg, response=self)
requests.exceptions.HTTPError: 404 Client Error: Not Found for url: http://localhost:8000/buckets/test-getvectors-1755937129/indexes/test-index/vectors:get

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/var/folders/st/cb03g5w149bdm_vtr9kltll40000gn/T/ipykernel_2788/2022115229.py", line 55, in <module>
    get_response = s3vectors_client.get_vectors(
        vectorBucketName=test_bucket,
    ...<3 lines>...
        returnMetadata=True
    )
  File "/Users/rkadeval/Desktop/work/s3vec_py/src/app/s3vectors_client.py", line 95, in get_vectors
    }
    
  File "/Users/rkadeval/Desktop/work/s3vec_py/src/app/s3v

In [14]:
# Test the new GetVectors API (fetch by keys)
print("🔧 Testing GetVectors API (batch key lookup)...")

# Test getting specific vectors by their keys
test_keys = ["doc1", "doc3", "doc5"]  # Get some specific documents

try:
    # Test the new GetVectors API endpoint
    print(f"\nTesting GetVectors with keys: {test_keys}")
    
    # This should work with our new GetVectors implementation
    get_response = s3vectors_client.get_vectors(
        vectorBucketName=bucket_name,
        indexName=index_name,
        keys=test_keys,
        returnData=True,
        returnMetadata=True
    )
    
    vectors = get_response.get('vectors', [])
    print(f"✅ GetVectors API works! Retrieved {len(vectors)} vectors")
    
    for vector in vectors:
        key = vector.get('key', 'Unknown')
        metadata = vector.get('metadata', {})
        has_data = 'data' in vector
        print(f"  - {key}: metadata={metadata}, has_data={has_data}")
        
    # Test getting non-existent keys
    print(f"\nTesting with non-existent keys...")
    missing_keys = ["nonexistent1", "missing2"]
    
    get_missing_response = s3vectors_client.get_vectors(
        vectorBucketName=bucket_name,
        indexName=index_name,
        keys=missing_keys,
        returnData=True,
        returnMetadata=True
    )
    
    missing_vectors = get_missing_response.get('vectors', [])
    print(f"✅ Non-existent keys handled correctly: found {len(missing_vectors)} vectors (expected 0)")
    
    # Test mixed keys (some exist, some don't)
    print(f"\nTesting with mixed keys (some exist, some don't)...")
    mixed_keys = ["doc1", "nonexistent", "doc2", "missing"]
    
    mixed_response = s3vectors_client.get_vectors(
        vectorBucketName=bucket_name,
        indexName=index_name,
        keys=mixed_keys,
        returnData=False,  # Test without data
        returnMetadata=True
    )
    
    mixed_vectors = mixed_response.get('vectors', [])
    print(f"✅ Mixed keys handled correctly: found {len(mixed_vectors)} vectors from {len(mixed_keys)} requested")
    
    for vector in mixed_vectors:
        key = vector.get('key', 'Unknown')
        has_data = 'data' in vector
        has_metadata = 'metadata' in vector
        print(f"  - {key}: has_data={has_data}, has_metadata={has_metadata}")

except Exception as e:
    print(f"❌ GetVectors API test failed: {e}")
    import traceback
    traceback.print_exc()

print("\n✅ GetVectors API testing completed!")

🔧 Testing GetVectors API (batch key lookup)...

Testing GetVectors with keys: ['doc1', 'doc3', 'doc5']
❌ GetVectors API test failed: Request failed: 404 Client Error: Not Found for url: http://localhost:8000/buckets/basic-demo-1755921004/indexes/demo-index/vectors:get

✅ GetVectors API testing completed!


Traceback (most recent call last):
  File "/Users/rkadeval/Desktop/work/s3vec_py/src/app/s3vectors_client.py", line 38, in _make_request
  File "/Users/rkadeval/Desktop/work/s3vec_py/.venv/lib/python3.13/site-packages/requests/models.py", line 1026, in raise_for_status
    raise HTTPError(http_error_msg, response=self)
requests.exceptions.HTTPError: 404 Client Error: Not Found for url: http://localhost:8000/buckets/basic-demo-1755921004/indexes/demo-index/vectors:get

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/var/folders/st/cb03g5w149bdm_vtr9kltll40000gn/T/ipykernel_2788/2540881915.py", line 12, in <module>
    get_response = s3vectors_client.get_vectors(
        vectorBucketName=bucket_name,
    ...<3 lines>...
        returnMetadata=True
    )
  File "/Users/rkadeval/Desktop/work/s3vec_py/src/app/s3vectors_client.py", line 95, in get_vectors
    }
    
  File "/Users/rkadeval/Desktop/work/s3vec_py/src/app/s3vector

In [ ]:
# Test Enhanced Filtering (new operators for AWS parity)
print("🔧 Testing Enhanced Filtering with new operators...")

# First, let's add some test data with numeric values for better filter testing
enhanced_documents = [
    {
        "key": "enhanced_doc1",
        "text": "Advanced machine learning techniques with high performance.",
        "metadata": {
            "category": "AI", 
            "topic": "machine_learning",
            "score": 95.5,
            "year": 2024,
            "tags": ["advanced", "performance"]
        }
    },
    {
        "key": "enhanced_doc2", 
        "text": "Basic programming concepts for beginners.",
        "metadata": {
            "category": "programming", 
            "topic": "basics",
            "score": 75.0,
            "year": 2023,
            "tags": ["basic", "tutorial"]
        }
    },
    {
        "key": "enhanced_doc3",
        "text": "High-performance computing and optimization.",
        "metadata": {
            "category": "computing", 
            "topic": "performance",
            "score": 88.7,
            "year": 2024,
            "tags": ["performance", "optimization"]
        }
    }
]

print("📦 Inserting enhanced test documents for filter testing...")
enhanced_vectors = []
for doc in enhanced_documents:
    embedding = get_text_embedding(doc['text'])
    enhanced_vectors.append({
        "key": doc['key'],
        "data": {"float32": embedding},
        "metadata": doc['metadata']
    })

try:
    response = s3vectors_client.put_vectors(
        vectorBucketName=bucket_name,
        indexName=index_name,
        vectors=enhanced_vectors
    )
    print(f"✅ Inserted {len(enhanced_vectors)} enhanced documents for filter testing")
except Exception as e:
    print(f"❌ Failed to insert enhanced documents: {e}")

# Test enhanced filtering capabilities
print("\n🔍 Testing Enhanced Filter Operators...")

# Test 1: Numeric comparison operators
print("\n1. Testing numeric comparison (score > 80)")
try:
    query_embedding = get_text_embedding("high performance systems")
    
    results = s3vectors_client.query_vectors(
        vectorBucketName=bucket_name,
        indexName=index_name,
        queryVector={"float32": query_embedding},
        topK=10,
        returnMetadata=True,
        filter={
            "operator": "greater_than",
            "metadata_key": "score",
            "value": 80
        }
    )
    
    vectors = results.get('vectors', [])
    print(f"  Found {len(vectors)} results with score > 80:")
    for result in vectors:
        metadata = result.get('metadata', {})
        score = metadata.get('score', 'N/A')
        key = result.get('key', 'Unknown')
        print(f"    - {key}: score={score}")
        
except Exception as e:
    print(f"  ❌ Numeric comparison test failed: {e}")

# Test 2: IN operator
print("\n2. Testing IN operator (category in ['AI', 'computing'])")
try:
    results = s3vectors_client.query_vectors(
        vectorBucketName=bucket_name,
        indexName=index_name,
        queryVector={"float32": query_embedding},
        topK=10,
        returnMetadata=True,
        filter={
            "operator": "in",
            "metadata_key": "category",
            "value": ["AI", "computing"]
        }
    )
    
    vectors = results.get('vectors', [])
    print(f"  Found {len(vectors)} results in ['AI', 'computing']:")
    for result in vectors:
        metadata = result.get('metadata', {})
        category = metadata.get('category', 'N/A')
        key = result.get('key', 'Unknown')
        print(f"    - {key}: category={category}")
        
except Exception as e:
    print(f"  ❌ IN operator test failed: {e}")

# Test 3: AND operator (combining conditions)
print("\n3. Testing AND operator (year = 2024 AND score >= 85)")
try:
    results = s3vectors_client.query_vectors(
        vectorBucketName=bucket_name,
        indexName=index_name,
        queryVector={"float32": query_embedding},
        topK=10,
        returnMetadata=True,
        filter={
            "operator": "and",
            "conditions": [
                {
                    "operator": "equals",
                    "metadata_key": "year",
                    "value": 2024
                },
                {
                    "operator": "greater_equal",
                    "metadata_key": "score",
                    "value": 85
                }
            ]
        }
    )
    
    vectors = results.get('vectors', [])
    print(f"  Found {len(vectors)} results with year=2024 AND score>=85:")
    for result in vectors:
        metadata = result.get('metadata', {})
        year = metadata.get('year', 'N/A')
        score = metadata.get('score', 'N/A')
        key = result.get('key', 'Unknown')
        print(f"    - {key}: year={year}, score={score}")
        
except Exception as e:
    print(f"  ❌ AND operator test failed: {e}")

# Test 4: OR operator
print("\n4. Testing OR operator (category = 'programming' OR score > 90)")
try:
    results = s3vectors_client.query_vectors(
        vectorBucketName=bucket_name,
        indexName=index_name,
        queryVector={"float32": query_embedding},
        topK=10,
        returnMetadata=True,
        filter={
            "operator": "or",
            "conditions": [
                {
                    "operator": "equals",
                    "metadata_key": "category",
                    "value": "programming"
                },
                {
                    "operator": "greater_than",
                    "metadata_key": "score",
                    "value": 90
                }
            ]
        }
    )
    
    vectors = results.get('vectors', [])
    print(f"  Found {len(vectors)} results with category='programming' OR score>90:")
    for result in vectors:
        metadata = result.get('metadata', {})
        category = metadata.get('category', 'N/A')
        score = metadata.get('score', 'N/A')
        key = result.get('key', 'Unknown')
        print(f"    - {key}: category={category}, score={score}")
        
except Exception as e:
    print(f"  ❌ OR operator test failed: {e}")

# Test 5: Complex nested filter
print("\n5. Testing complex nested filter")
try:
    results = s3vectors_client.query_vectors(
        vectorBucketName=bucket_name,
        indexName=index_name,
        queryVector={"float32": query_embedding},
        topK=10,
        returnMetadata=True,
        filter={
            "operator": "and",
            "conditions": [
                {
                    "operator": "or",
                    "conditions": [
                        {"operator": "equals", "metadata_key": "category", "value": "AI"},
                        {"operator": "equals", "metadata_key": "category", "value": "computing"}
                    ]
                },
                {
                    "operator": "greater_than",
                    "metadata_key": "score",
                    "value": 80
                }
            ]
        }
    )
    
    vectors = results.get('vectors', [])
    print(f"  Found {len(vectors)} results with complex filter:")
    for result in vectors:
        metadata = result.get('metadata', {})
        category = metadata.get('category', 'N/A')
        score = metadata.get('score', 'N/A')
        key = result.get('key', 'Unknown')
        print(f"    - {key}: category={category}, score={score}")
        
except Exception as e:
    print(f"  ❌ Complex filter test failed: {e}")

print("\n✅ Enhanced filtering tests completed!")